In [ ]:
# Importing the necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_absolute_error, accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split 
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [ ]:
# Reading the files.

df_train = pd.read_csv("../input/ventilator-pressure-prediction/train.csv")
df_test = pd.read_csv("../input/ventilator-pressure-prediction/test.csv")
df_sub = pd.read_csv("../input/ventilator-pressure-prediction/sample_submission.csv")

In [ ]:
#def add_features(df):
#    df['area'] = df['time_step'] * df['u_in']
#    df['area'] = df.groupby('breath_id')['area'].cumsum()
#    df['cross']= df['u_in']*df['u_out']
#    df['cross2']= df['time_step']*df['u_out']
#    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
#    df['one'] = 1
#    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
#    df['u_in_cummean'] =df['u_in_cumsum'] /df['count']
#    df['breath_id_lag']=df['breath_id'].shift(1).fillna(0)
#    df['breath_id_lag2']=df['breath_id'].shift(2).fillna(0)
#    df['breath_id_lagsame']=np.select([df['breath_id_lag']==df['breath_id']],[1],0)
#    df['breath_id_lag2same']=np.select([df['breath_id_lag2']==df['breath_id']],[1],0)
#    df['u_in_lag'] = df['u_in'].shift(1).fillna(0)
#    df['u_in_lag'] = df['u_in_lag']*df['breath_id_lagsame']
#    df['u_in_lag2'] = df['u_in'].shift(2).fillna(0)
#    df['u_in_lag2'] = df['u_in_lag2']*df['breath_id_lag2same']
#    df['u_out_lag2'] = df['u_out'].shift(2).fillna(0)
#    df['u_out_lag2'] = df['u_out_lag2']*df['breath_id_lag2same']
#    return df

#train_x = add_features(train_x)
#test = add_features(df_test)

n_steps = 80

# dividing the data into features and target variable.
train_x = df_train.drop(columns=["pressure", "id", "breath_id"])
test = df_test.drop(columns=["id", "breath_id"])

#scalar = MinMaxScaler()
#train_x = scalar.fit_transform(train_x)
#test_19 = scalar.fit_transform(test_19)

train_y = df_train["pressure"].to_numpy().reshape(-1, 80)
train_x = train_x.to_numpy().reshape(-1, 80, train_x.shape[-1])
test = test.to_numpy().reshape(-1, 80, test.shape[-1])

In [ ]:
# Create Bi-LSTM model

def get_model():
    act = "selu"
    model = keras.Sequential([
        layers.InputLayer(input_shape=(80,5)),
        layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
        layers.Bidirectional(layers.LSTM(256, return_sequences=True)),
        layers.Bidirectional(layers.LSTM(1028, return_sequences=True)),
        #layers.Bidirectional(layers.LSTM(2048, return_sequences=True)),
        #layers.Dropout(0.2),
        #layers.Flatten(),
        #layers.Dense(128, activation=act),
        #layers.Dense(256, activation=act),
        #layers.Dense(50, activation=act),
        layers.Dense(1)
    ])
    return model


model = get_model()
model.compile(optimizer="adam", loss="mae")

model.summary()

In [ ]:
# Fit Bi-LSTM

def fit_model(mod):
    early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                               patience = 5)
    history = mod.fit(train_x, train_y, epochs = 100,
                        validation_split = 0.3, batch_size = 32, 
                        shuffle = False, callbacks = [early_stop]
                       )
    return history

history_bilstm = fit_model(model)

In [ ]:
prediction = model.predict(test)

pred = prediction.flatten()

sns.displot(pred)

In [ ]:
def sub(test_prediction):
    df = pd.DataFrame({"id":df_test.id, "pressure":test_prediction})
    return df

submission = sub(pred)

submission.shape

In [ ]:
# saving the model

model.save("Bi-LSTM-1-val_loss: 0.2526.h5")

# Ensemble 

In [ ]:
#model1 = keras.models.load_model("../input/google-brain-ventilator-models/Bi-LSTM-1-val_loss_ 0.315.h5") #19
#model2 = keras.models.load_model("../input/google-brain-ventilator-models/Bi-LSTM-1-val_loss_ 0.38.h5") #19
#model3 = keras.models.load_model("../input/google-brain-ventilator-models/Bi-LSTM-1-val_loss_ 0.29.h5") #9
model4 = keras.models.load_model("../input/google-brain-ventilator-models/Bi-LSTM-1-val_loss_ 0.2526.h5") #9
model5 = keras.models.load_model("../input/google-brain-ventilator-models/Bi-LSTM-val_loss_ 0.2644.h5") #9

#prediction1 = model1.predict(test)
#prediction2 = model2.predict(test)
prediction4 = model4.predict(test)
prediction5 = model5.predict(test)

#prediction1 = prediction1.flatten()
#prediction2 = prediction2.flatten()
prediction4 = prediction4.flatten()
prediction5 = prediction5.flatten()

final_prediction = (prediction4+prediction5)/2

sns.displot(final_prediction)

In [ ]:
# creating the submission file.

def sub(test_prediction):
    df = pd.DataFrame({"id":df_test.id, "pressure":test_prediction})
    return df

submission = sub(final_prediction)

submission.shape

In [ ]:
submission.to_csv("submission_ensemble_2_models_final_submission.csv", index=False)